<a href="https://colab.research.google.com/github/abdvs/SQL-analysis/blob/main/Automobile_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=c73f90a961b286792f66eff298d9cfff20c01143fe236139fa5bf7daf539a0d6
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [3]:
import pandas as pd
import sqlite3

In [5]:
df = pd.read_csv('/content/automobile_data.csv',encoding='unicode_escape')

In [6]:
conn = sqlite3.connect('auto.db')


In [7]:
df.to_sql('auto',conn,if_exists = 'replace', index = False)
conn.close()

In [8]:
conn = sqlite3.connect('auto.db')
conn.commit()
users =  pd.read_sql_query('''select * from auto''',conn)

In [ ]:
pd.read_sql_query('''select * from auto''',conn)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
201,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
202,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
203,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.4,23.0,106,4800,26,27,22470


**What is the most common body style among the vehicles**

In [ ]:
pd.read_sql_query('''

select [body-style],count(*) as style_count from auto
where price is not null and bore is not null and  stroke is not null and horsepower is not null and [peak-rpm] is not null
group by [body-style]
order by count(*) desc



''',conn)

,body-style,style_count
0,sedan,96
1,hatchback,70
2,wagon,25
3,hardtop,8
4,convertible,6


###What is the average length of cars with a fuel type of 'gas'

In [ ]:
pd.read_sql_query('''

select [fuel-type],round(avg(length),2) as avg_len from auto
where price is not null and bore is not null and  stroke is not null and horsepower is not null and [peak-rpm] is not null
and [fuel-type]='gas'
group by [fuel-type]
order by avg(length) desc



''',conn)

,fuel-type,avg_len
0,gas,173.19


**How many cars have a curb weight greater than 2500 pounds and city mileage greater than 20 mpg?**

In [ ]:
pd.read_sql_query('''


select make, count(*) as count from auto
where [curb-weight]>2500 and [city-mpg]>20
group by make
order by count(*) desc



''',conn)

,make,count
0,toyota,7
1,volvo,6
2,peugot,5
3,saab,4
4,mercedes-benz,4
5,volkswagen,2
6,subaru,2
7,bmw,2
8,alfa-romero,2
9,renault,1


**How many cars have a wheel base greater than 100 inches and highway mileage less than 25 mpg?**

In [ ]:
pd.read_sql_query('''


select make,count(*) as no_of_cars from auto
where [wheel-base]>100 and [highway-mpg]<25
group by make
order by count(*) desc


''',conn)

,make,no_of_cars
0,peugot,6
1,toyota,4
2,volvo,3
3,mercedes-benz,3
4,jaguar,3
5,bmw,3
6,nissan,2
7,volkswagen,1
8,mercury,1
9,audi,1


**What is the avg price of cars with an aspiration of 'std' and engine type 'ohc'?**

In [ ]:
pd.read_sql_query('''


select aspiration,[engine-type] ,round(avg(price),2) as avg_price from auto
where aspiration ='std' and [engine-type]='ohc'
group by aspiration,[engine-type]

''',conn)

,aspiration,engine-type,avg_price
0,std,ohc,10398.78


**Identify the top 5 car makes with the highest average price, excluding any makes with less than 5 entries in the dataset.**

In [ ]:
pd.read_sql_query('''


select make,round(avg(price),2) as avg_price from auto
group by make
having count(*) >=5
order by avg(price) desc

''',conn)

,make,avg_price
0,mercedes-benz,33647.00
1,bmw,26118.75
2,porsche,25120.40
3,volvo,18063.18
4,peugot,15489.09
5,audi,15307.86
6,saab,15223.33
7,mazda,10652.88
8,nissan,10415.67
9,volkswagen,10077.50


**What is the average price of vehicles grouped by make and fuel type?**


In [ ]:
pd.read_sql_query('''


select make, [fuel-type],round(avg(price),2) as avg_price from auto
group by make, [fuel-type]
order by round(avg(price),2) desc
''',conn)

,make,fuel-type,avg_price
0,mercedes-benz,gas,38900.00
1,jaguar,gas,34600.00
2,mercedes-benz,diesel,28394.00
3,bmw,gas,26118.75
4,porsche,gas,25120.40
5,volvo,diesel,22470.00
6,volvo,gas,17622.50
7,mercury,gas,16503.00
8,peugot,diesel,15797.00
9,alfa-romero,gas,15498.33


from matplotlib import pyplot as plt
_df_0['avg_price'].plot(kind='hist', bins=20, title='avg_price')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('fuel-type').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['avg_price'].plot(kind='line', figsize=(8, 4), title='avg_price')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_3['fuel-type'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_3, x='avg_price', y='fuel-type', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

**Calculate the average city and highway mileage for vehicles grouped by body style and fuel type.**


In [ ]:
pd.read_sql_query('''

select [body-style],[fuel-type] ,round(avg([city-mpg]),2) as city_ml,round(avg([highway-mpg]),2) as high_ml from auto
group by [body-style],[fuel-type]
order by round(avg([city-mpg]),2),round(avg([highway-mpg]),2) desc

''',conn)

,body-style,fuel-type,city_ml,high_ml
0,convertible,gas,20.50,26.00
1,hardtop,gas,21.57,27.57
2,hardtop,diesel,22.00,25.00
3,wagon,diesel,24.00,25.00
4,wagon,gas,24.05,29.23
5,sedan,gas,24.16,29.78
6,hatchback,gas,26.14,31.96
7,sedan,diesel,31.60,36.53
8,hatchback,diesel,38.00,47.00


**Max RPM according to drive wheels**

In [12]:
pd.read_sql_query('''
select [drive-wheels],max([peak-rpm]) max_rpm from auto
where [peak-rpm] !='?'
group by [drive-wheels]
order by max_rpm desc
''',conn)

,drive-wheels,max_rpm
0,rwd,6600
1,fwd,6000
2,4wd,5500


Insights:


*   Mercedes and BMW makes tend to have higher average prices, potentially indicating brand perception.

*   Dodge make has lowest average price which has scope for better marketing.


*  Hatchback type with diesel fuel type has high  Average city and highway mileage
*   Gas fuel type is expensive than diesel

* Porsche make with Gas type has maximum RPM
* Rear Wheel drive has max peak RPM

